## NER - Natasha

Документация yargy: https://yargy.readthedocs.io/ru/latest/reference.html

Лучше всего с такими штуками разбираться на практике. Давайте попробуем написать правила для извлечения персон. Каждая персона должна описываться 3 полями - Имя, Фамилия, Отчество. Также у Персоны должны быть атрибуты - должность и место работы. 

За основу возьмем пример из документации:

In [27]:
sents = open('sents.txt').read().splitlines()

In [34]:
sents

['Напомним, что премьер-министр России Дмитрий Медведев подписал изменения в Правилах дорожного движения.',
 'В секте Асахары «министр» Ниими слыл страшным человеком.',
 'С тайной полицией Асахары был связан еще один из семи этапированных накануне смертников – Кийохиде Хаякава, «министр созидания».',
 'По словам Косуги, накануне «министр» Хаякава в личной беседе поблагодарил его за помощь «Аум» и попросил сделать кое-что для «спасения человечества», а именно – убить шефа полиции Японии.',
 'Как только что сообщило немецкое государственное информационное агентство "Deutsche Welle", видный немецкий политик от СДПГ, в прошлом премьер-министр земли Бранденбург Маттиас Платцек, в интервью радиостанции "Deutschlandfunk" в пятницу 16 марта заявил, что по его сведениям лаборатория, которая занималась исследованиями боевых отравляющих веществ в советское время находилась на территории нынешнего Узбекистана.',
 'Между тем вице-премьер РМ по евроинтеграции Юрий Лянкэ заявил накануне на румынском 

In [128]:
from yargy import Parser, rule
from yargy.predicates import gram, eq
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from IPython.display import display

Person = fact(
    'Person',
    ['position', 'name']
)
Name = fact(
    'Name',
    ['first', 'last']
)

positions = [
    'премьер-министр',
    'президент',
    'премьер',
    'вице-премьер',
    'председатель',
    'мэр',
    'министр',
    'вице-канцлер',
    'канцлер',
    'депутат',
    'представитель',
    'начальник',
    'сотрудник',
    'глава',
    'руководитель',
    'член',
    'директор'
    
]

ex_positions = ['экс-' + pos for pos in positions]

POSITION = morph_pipeline(positions + ex_positions)

NAME = rule(
    gram('Name').interpretation(
        Name.first.inflected()
    ),
    gram('Surn').interpretation(
        Name.last.inflected()
    )
).interpretation(
    Name
)

PERSON = rule(
    POSITION.interpretation(
        Person.position.inflected()
    ),
    gram('gent').optional().repeatable(),
    eq('и').optional(),
    gram('gent').optional().repeatable(),
    gram('LATN').optional(),
    gram('Geox').optional(),
    gram('Orgn').optional(),
    gram('Abbr').optional(),
    NAME.interpretation(
        Person.name
    )
).interpretation(
    Person
)


parser = Parser(PERSON)

Посмотрим, что получается:

In [129]:
matches = []

for sent in sents:
    for match in parser.findall(sent):
        matches.append(match)

In [130]:
len(matches)

52